In [1]:

# Liquidity math adapted from:
#   https://github.com/Uniswap/uniswap-v3-periphery/blob/main/contracts/libraries/LiquidityAmounts.sol

# formula ref：
# http://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf

# 
'''
Computes the amount of liquidity received for a given amount of token0 and price range
sa: sqrtRatioAX96 ticket下限
sb: sqrtRatioBX96 ticket上限
x: 存入的x token的数量
'''

def get_liquidity_0(x, sa, sb):
    return x * sa * sb / (sb - sa)

def get_liquidity_1(y, sa, sb):
    return y / (sb - sa)

'''
sp 是现在的价格 计算流动性的时候分为三种情况：
1. 现在的价格比tick的下限还低 说明流动性全是以x token来提供的 只要算x提供了多少流动性就可以
2. 现在的价格在tick之内 计算流动性的时候是两者之间较小的一个
3. 现在的价格比tick的上限还高 说明流动性全是以y token来提供的 只要计算y提供了多少流动性就可以
'''
def get_liquidity(x, y, sp, sa, sb):
    if sp <= sa:
        liquidity = get_liquidity_0(x, sa, sb)
    elif sp < sb:
        liquidity0 = get_liquidity_0(x, sp, sb)
        liquidity1 = get_liquidity_1(y, sa, sp)
        liquidity = min(liquidity0, liquidity1)
    else:
        liquidity = get_liquidity_1(y, sa, sb)
    return liquidity

In [2]:

# Calculate x and y given liquidity and price range

'''
这个计算公式就是formulas/calculate-v3-liquidity.PNG里推导出来的
如果现在的价格在a和b之间 sp为sp
如果现在的价格比a小 sp为sa
如果现在的价格比b大 sp为pb
总之就是如果现在的价格不在所选的价格区间内 就选顶端的其中一个
'''

def calculate_x(L, sp, sa, sb): 
    # if the price is outside the range, use the range endpoints instead
    sp = max(min(sp, sb), sa)
    return L * (sb - sp) / (sp * sb)  #from eq(12) in Tech Note

def calculate_y(L, sp, sa, sb):
    # if the price is outside the range, use the range endpoints instead
    sp = max(min(sp, sb), sa)
    return L * (sp - sa)  #from eq(13) in Tech Note
     

In [3]:

# 这个和上一个块用的是同一个公式
# 就是带入其他值算pa 和pb
# Two different ways how to calculate p_a. calculate_a1() uses liquidity as an input, calculate_a2() does not.

def calculate_a1(L, sp, sb, x, y):
    # https://www.wolframalpha.com/input/?i=solve+L+%3D+y+%2F+%28sqrt%28P%29+-+a%29+for+a
    # sqrt(a) = sqrt(P) - y / L
    return (sp - y / L) ** 2  #from eq(14) in Tech Note

def calculate_a2(sp, sb, x, y):
    # https://www.wolframalpha.com/input/?i=solve+++x+sqrt%28P%29+sqrt%28b%29+%2F+%28sqrt%28b%29++-+sqrt%28P%29%29+%3D+y+%2F+%28sqrt%28P%29+-+a%29%2C+for+a
    # sqrt(a) = (y/sqrt(b) + sqrt(P) x - y/sqrt(P))/x
    #    simplify:
    # sqrt(a) = y/(sqrt(b) x) + sqrt(P) - y/(sqrt(P) x)
    sa = y / (sb * x) + sp - y / (sp * x)  #from eq(16) in Tech Note
    return sa ** 2

# Two different ways how to calculate p_b. calculate_b1() uses liquidity as an input, calculate_b2() does not.

def calculate_b1(L, sp, sa, x, y):
    # https://www.wolframalpha.com/input/?i=solve+L+%3D+x+sqrt%28P%29+sqrt%28b%29+%2F+%28sqrt%28b%29+-+sqrt%28P%29%29+for+b
    # sqrt(b) = (L sqrt(P)) / (L - sqrt(P) x)
    return ((L * sp) / (L - sp * x)) ** 2  #from eq(15) in Tech Note

def calculate_b2(sp, sa, x, y):
    # find the square root of b:
    # https://www.wolframalpha.com/input/?i=solve+++x+sqrt%28P%29+b+%2F+%28b++-+sqrt%28P%29%29+%3D+y+%2F+%28sqrt%28P%29+-+sqrt%28a%29%29%2C+for+b
    # sqrt(b) = (sqrt(P) y)/(sqrt(a) sqrt(P) x - P x + y)
    P = sp ** 2
    return (sp * y / ((sa * sp - P) * x + y)) ** 2  #from eq(17) in Tech Note
     

In [ ]:

# Calculating c and d

def calculate_c(p, d, x, y):
    return y / ((d - 1) * p * x + y)  #from eq(24) in Tech Note

def calculate_d(p, c, x, y):
    return 1 + y * (1 - c) / (c * p * x)  #from eq(25) in Tech Note